In [23]:
PROJECT="myfirstproject-226013"

In [94]:
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage
import pandas as pd

# Make clients.
bqclient = bigquery.Client(project=PROJECT)
bqstorageclient = bigquery_storage.BigQueryReadClient()
query_string = """
SELECT * from telco.churn
"""

df = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)

In [95]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,9732-OUYRN,Female,0,True,False,49,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,918.7,False
1,0661-KQHNK,Female,0,True,True,6,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,105.5,False
2,4709-LKHYG,Female,0,True,True,29,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Electronic check,20.0,540.05,False
3,9824-QCJPK,Male,0,True,False,36,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,666.75,False
4,4716-MRVEN,Female,0,False,False,29,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,599.3,False


In [99]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df=df.fillna(df.mean())

In [131]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Pclass: passenger class
# Parch: parents and children
BINARY_FEATURES = ['gender',
            'SeniorCitizen',
            'Partner',
            'Dependents',
            'PhoneService',
            'MultipleLines',
            'PaperlessBilling']

NUMERIC_FEATURES = [
            'tenure',
            'MonthlyCharges',
            'TotalCharges']

CATEGORICAL_FEATURES = [
            'InternetService',
            'OnlineSecurity',
            'DeviceProtection',
            'TechSupport',
            'StreamingTV',
            'StreamingMovies',
            'Contract',
            'PaymentMethod']

# all rows but only selected features/columns

X = df.loc[:, BINARY_FEATURES+NUMERIC_FEATURES+CATEGORICAL_FEATURES]

y = df.Churn


In [129]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV


binary_transformer = OneHotEncoder(sparse=False)
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')


preprocessor = ColumnTransformer(
    transformers=[
        ('bin', binary_transformer, BINARY_FEATURES),
        ('num', numeric_transformer, NUMERIC_FEATURES),
        ('cat', categorical_transformer, CATEGORICAL_FEATURES)])


# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC(kernel='linear'))])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [130]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.83      0.89      0.86      1027
        True       0.64      0.52      0.57       382

    accuracy                           0.79      1409
   macro avg       0.73      0.70      0.72      1409
weighted avg       0.78      0.79      0.78      1409

[[913 114]
 [183 199]]
